In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##Import Data
df=pd.read_csv('/kaggle/input/hr-analytics/HR_comma_sep.csv')
subdf = df.copy()
subdf.head()

In [ ]:
df["promotion_last_5years"].value_counts()

In [ ]:
df["left"].value_counts()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
len(df.loc[df['left'] == 1])

In [ ]:
left_Employee =len(df) -  len(df.loc[df['left'] == 0]) 
print(left_Employee)
not_promoted = len(df) -  len(df.loc[df['promotion_last_5years'] == 0]) 
print(not_promoted)

In [ ]:
df.loc[df['left'] == 0].describe()

In [ ]:
equiv = {1:"Resignees", 0:"Stayed"}
df["left"] = df["left"].map(equiv)

In [ ]:

equiv = {1:"Promoted", 0:"Not Promoted"}
df["promotion_last_5years"] = df["promotion_last_5years"].map(equiv)

In [ ]:
df["promotion_last_5years"]

In [ ]:
df.groupby('left').mean()

In [ ]:
pd.crosstab(df.salary,df.left).plot(kind='bar')

In [ ]:
pd.crosstab(df.Department,df.left).plot(kind='bar')

In [ ]:
pd.crosstab(df.promotion_last_5years,df.left).plot(kind='bar')

In [ ]:
pd.crosstab(df.promotion_last_5years,df.left)

In [ ]:
subdf.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:

#Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    #score = model.score(X_valid,y_valid)
    print(score)
    return X_train.shape
    return mean_absolute_error(y_valid, preds)
# def score_dataset(X_train, X_valid, y_train, y_valid):
#     model = RandomForestRegressor(n_estimators=100, random_state=0)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_valid)
#     print(model.score(X_valid,y_valid))
#     return mean_absolute_error(y_valid, preds)

In [ ]:
y = subdf.left
X = subdf.drop(['left'], axis=1)

In [ ]:
X.info()
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [ ]:
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
numerical_cols

In [ ]:
# Get list of categorical variables
s = (X_train_full.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

In [ ]:
my_cols = object_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
X_train["Department"].value_counts()

In [ ]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Apply one-hot encoder to each column with categorical data
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_train

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [ ]:
OH_X_train.info()

In [ ]:
print("MAE from Approach 3 (One-Hot Encoding):") 
x = score_dataset(OH_X_train, OH_X_valid, y_train, y_valid)
print(x)